In [1]:
from pprint import pprint
from IPython.display import display, Markdown

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import get_scorer, confusion_matrix
from sklearn.metrics import f1_score, precision_score, roc_auc_score

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import make_column_transformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder

In [2]:
pd.options.display.float_format = '{:.4f}'.format

In [3]:
def title(title, n=3):
    return Markdown('{} {}'.format('#'*(max(n%4, 1)), title))

In [4]:
# !ls -lah

In [5]:
# filename = 'KaggleV2-May-2016.csv'
filename = 'https://github.com/dm6801/noshow_dataset/raw/master/KaggleV2-May-2016.csv'

In [6]:
# !wc -l "$filename"
# !head -n2 "$filename"

#### Variables' description:

    PatientId - identification of a patient;
    AppointmentID - identification of each appointment;
    Gender - male or female;
    ScheduledDay - day when a patient registered for an appointment;
    AppointmentDay - day of actuall appointment;
    Age - patient age;
    Neighbourhood - where the appointment takes place (hospital location);
    Scholarship - 1 for True, 0 for False. For more details read the article (the link is here: https://bit.ly/3AYv4GF);
    Hipertension - 1 for True, 0 for False;
    Diabetes - 1 for True, 0 for False;
    Alcoholism - 1 for True, 0 for False;
    Handcap - number of disabilities a patient has;
    SMS_received - 1 if one or more messages were sent to the patient;
    No-show - 'No' if the patient showed up to their appointment, 'Yes' if they did not show up.

In [7]:
df = pd.read_csv(filename)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   PatientId       110527 non-null  float64
 1   AppointmentID   110527 non-null  int64  
 2   Gender          110527 non-null  object 
 3   ScheduledDay    110527 non-null  object 
 4   AppointmentDay  110527 non-null  object 
 5   Age             110527 non-null  int64  
 6   Neighbourhood   110527 non-null  object 
 7   Scholarship     110527 non-null  int64  
 8   Hipertension    110527 non-null  int64  
 9   Diabetes        110527 non-null  int64  
 10  Alcoholism      110527 non-null  int64  
 11  Handcap         110527 non-null  int64  
 12  SMS_received    110527 non-null  int64  
 13  No-show         110527 non-null  object 
dtypes: float64(1), int64(8), object(5)
memory usage: 11.8+ MB


In [9]:
df.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,29872499824296.0000,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,558997776694438.0000,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4262962299951.0000,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,867951213174.0000,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8841186448183.0000,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


In [10]:
df['No-show'] = df['No-show'].map(dict(No=0, Yes=1))

In [11]:
df['Gender'] = df['Gender'].map(dict(F=0, M=1))

In [12]:
(lambda x:x[x.dt.days<0].dt.days
)(pd.to_datetime(df['AppointmentDay']).dt.date - pd.to_datetime(df['ScheduledDay']).dt.date)

27033   -1
55226   -1
64175   -1
71533   -6
72362   -1
dtype: int64

In [13]:
df.drop((lambda x:x[x.dt.days<0].dt.days
)(pd.to_datetime(df['AppointmentDay']).dt.date - pd.to_datetime(df['ScheduledDay']).dt.date).index, inplace=True)

In [14]:
X = df.drop('No-show', axis=1)
y = df['No-show']

In [15]:
def cyclicalEncode(x, end, start=0, names={}):
    
    sin = np.sin(2 * np.pi * (x+float(start))/end)
    if 'sin' in names:
        sin.rename(names['sin'], inplace=True)
        
    cos = np.cos(2 * np.pi * (x+float(start))/end)
    if 'cos' in names:
        cos.rename(names['cos'], inplace=True)
        
    return sin, cos

In [16]:
class DebugTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self, name='', verbose=[], **kwargs):
        self.name = name if name else self.__class__.__name__
        self.tag = f'{self.name:<30}'
        self.verbose = verbose
        self.col_tf = kwargs.get('col_tf')
        
        
    def fit(self, X, y=None):
        print(self.tag, 'fit',
              '\t\tX shape:', X.shape, 
              '\ty shape:', y.shape if isinstance(y, pd.Series) else None)
        
        # if 'fit' in self.verbose:
        #     display(X if isinstance(X, pd.DataFrame) else pd.DataFrame(X),
        #             y if isinstance(y, pd.Series) else pd.Series(y) if y != None else None)
        #     print()
        
        return self
    

    def transform(self, X, y=None):
        print(self.tag, 'transform', 
              '\tX shape:', X.shape, 
              '\ty shape:', y.shape if isinstance(y, pd.Series) else None)
        
        if 'transform' in self.verbose:
            columns = self.col_tf.get_feature_names_out() if self.col_tf != None else None
            
            if not isinstance(X, pd.DataFrame):
                _X = pd.DataFrame(X, columns=columns)
            elif columns != None:
                _X = X.copy()
                _X.columns = columns
            else:
                _X = X
                
            display(_X,
                    y if isinstance(y, pd.Series) else pd.Series(y) if y != None else None)
            
            display(_X.describe().T)
            if isinstance(y, pd.Series):
                display(y.describe())
            elif y != None:
                display(pd.Series(y).describe())
                
            print()
        
        return X

In [17]:
class FeaturerEngineering(BaseEstimator, TransformerMixin):
    
    def __init__(self, name=''):
        self.name = name if name else self.__class__.__name__
        
        
    def fit(self, X, y=None):
        # print(f'{self.name:<30}', 'fit',
        #       '\t\tX shape:', X.shape, 
        #       '\ty shape:', y.shape if isinstance(y, pd.Series) else '')
        
        self.neighbourhood_ratio = (X.join(y)
                                    .pivot_table(
                                        index='Neighbourhood',
                                        columns=y.name,
                                        values='Age',
                                        aggfunc='count')
                                    .apply(lambda x:x[1]/x[0], axis=1))
        
        self.previous_no_show_ratio = (X.join(y)
                                       .pivot_table(
                                           index='PatientId',
                                           columns=y.name,
                                           values='Age',
                                           aggfunc='count')
                                    .apply(lambda x:x[1]/x[0], axis=1))
        
        return self
    

    def transform(self, X, y=None):
        # print(f'{self.name:<30}', 'transform', 
        #       '\tX shape:', X.shape, 
        #       '\ty shape:', y.shape if isinstance(y, pd.Series) else '')
        
        neighbourhood = (X['Neighbourhood']
                         .map(self.neighbourhood_ratio)
                         .fillna(self.neighbourhood_ratio.median())
                         .rename('neighbourhood'))
        
        previous_no_show = (X['PatientId']
                            .map(self.previous_no_show_ratio)
                            .fillna(self.previous_no_show_ratio.median())
                            .rename('previous_no_show'))
        
        scheduled_datetime = pd.to_datetime(X['ScheduledDay'])
        appointmt_datetime = pd.to_datetime(X['AppointmentDay'])
        
        days_diff = (appointmt_datetime.dt.date
                     - scheduled_datetime.dt.date).dt.days.rename('days_diff')
        
        scheduled_day_of_week = scheduled_datetime.dt.dayofweek.rename('scheduled_day')
        scheduled_hour = scheduled_datetime.dt.hour.rename('scheduled_hour')
        
        appointmt_day_of_week = appointmt_datetime.dt.dayofweek.rename('appointment_day')
        
#         scheduled_month_sin, \
#         scheduled_month_cos = cyclicalEncode(scheduled_datetime.dt.month, 12, 
#                                              names={'sin': 'scheduled_month_sin',
#                                                     'cos': 'scheduled_month_cos'})
        
#         scheduled_day_sin, \
#         scheduled_day_cos = cyclicalEncode(scheduled_datetime.dt.dayofweek, 7, 
#                                            names={'sin': 'scheduled_day_sin',
#                                                   'cos': 'scheduled_day_cos'})
        
#         scheduled_hour_sin, \
#         scheduled_hour_cos = cyclicalEncode(scheduled_datetime.dt.hour, 24, 
#                                             names={'sin': 'scheduled_hour_sin',
#                                                    'cos': 'scheduled_hour_cos'})
        
#         appointmt_month_sin, \
#         appointmt_month_cos = cyclicalEncode(appointmt_datetime.dt.month, 12, 
#                                              names={'sin': 'appointmt_month_sin',
#                                                     'cos': 'appointmt_month_cos'})
        
#         appointmt_day_sin, \
#         appointmt_day_cos = cyclicalEncode(appointmt_datetime.dt.dayofweek, 7, 
#                                            names={'sin': 'appointmt_day_sin',
#                                                   'cos': 'appointmt_day_cos'})
        
        return X.join([
            days_diff,
            neighbourhood,
            previous_no_show,
            scheduled_day_of_week,
            scheduled_hour,
            appointmt_day_of_week,
#             scheduled_month_sin,
#             scheduled_month_cos,
#             scheduled_day_sin,
#             scheduled_day_cos,
#             scheduled_hour_sin,
#             scheduled_hour_cos,
#             appointmt_month_sin,
#             appointmt_month_cos,
#             appointmt_day_sin, 
#             appointmt_day_cos,
        ])

In [18]:
df.columns

Index(['PatientId', 'AppointmentID', 'Gender', 'ScheduledDay',
       'AppointmentDay', 'Age', 'Neighbourhood', 'Scholarship', 'Hipertension',
       'Diabetes', 'Alcoholism', 'Handcap', 'SMS_received', 'No-show'],
      dtype='object')

In [19]:
def my_pipeline(estimator, remainder='drop', verbose=0):
    
    column_transformer = make_column_transformer(
#             (StandardScaler(), [

#             ]),
            (MinMaxScaler(), [
                'Handcap',
                'previous_no_show',
                'Age',
                'days_diff',
                'neighbourhood',
                'scheduled_day',
                'scheduled_hour',
                'appointment_day',
                'SMS_received',
                'Scholarship',
                'Hipertension',
                'Diabetes',
                'Alcoholism',
#                 'scheduled_month_sin',
#                 'scheduled_month_cos',
#                 'scheduled_day_sin',
#                 'scheduled_day_cos',
#                 'scheduled_hour_sin',
#                 'scheduled_hour_cos',
#                 'appointmt_month_sin',
#                 'appointmt_month_cos',
#                 'appointmt_day_sin', 
#                 'appointmt_day_cos',
            ]),
#             (OneHotEncoder(drop='first'), [
#                 'Gender',
#             ]),
#             ('passthrough', [
#                 'SMS_received',
#                 'Scholarship',
#                 'Hipertension',
#                 'Diabetes',
#                 'Alcoholism',
#             ]),
            remainder=remainder,
            n_jobs=-1)
    
    if verbose:
        steps = [
            DebugTransformer('before'),
            FeaturerEngineering(),
            DebugTransformer('after FeaturerEngineering', ['transform'] if verbose==2 else []),
            column_transformer,
            DebugTransformer('after ColumnTransformer', ['transform'] if verbose==2 else [],
                             col_tf=column_transformer),
        ]
    else:
        steps = [
            FeaturerEngineering(),
            column_transformer,
        ]
    
    return make_pipeline(*steps, estimator)

In [20]:
def estimate(estimator, X, y, scoring=None, remainder='drop', verbose=0, *args, **kwargs):
    
    
    # verbose
    def _print(_title, func, *args, **kwargs):
        if verbose: display(title(_title))
        out = func(*args, **kwargs)
        if verbose: print()
        return out
    
    
    # create pipeline
    if isinstance(estimator, Pipeline):
        pipeline = estimator
    else:
        pipeline = my_pipeline(estimator, remainder, verbose)
    display(title(pipeline.steps[-1][1].__class__.__name__, 2))
    pprint([s[1] for s in pipeline.steps], width=1)
    
    
    # split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123,
                                                        test_size=.2, shuffle=True, stratify=y)
    
    
    # cross validate
    cross = cross_validate(pipeline, X_train, y_train, scoring=scoring, n_jobs=-1)
    display(title('cross_validate'))
    pprint({k: round(v.mean(), 3) for k, v in cross.items()}, width=1)
    print()

    
    # fit, predict
    _print('fitting', pipeline.fit, X_train, y_train)
    y_pred = _print('predeicting', pipeline.predict, X_test)
    y_proba = _print('predicting probabilities', pipeline.predict_proba, X_test)[:, 1]
    
    
    # confusion matrix
    display(title('confusion matrix'))
    conf_mtx = confusion_matrix(y_test, y_pred)
    pprint(conf_mtx, width=1)
    print()
    
    
    # scores
    display(title('scores'))
    if isinstance(scoring, str):
        score_func = get_scorer(scoring)._score_func
        print(score_func.__name__ , round(score_func(y_test, y_pred), 3))
    else:
        scores = {f.__name__: 
                  f(y_test, y_proba) if 'y_score' in f.__code__.co_varnames else f(y_test, y_pred)
                  for f in map(lambda x:get_scorer(x)._score_func, scoring)}
        pprint({k: round(v, 3) for k, v in scores.items()}, width=1)

In [21]:
# explicit pipeline

estimate(make_pipeline(StandardScaler(),
                       RandomForestClassifier(random_state=100, class_weight='balanced',
                                              n_estimators=100, max_depth=13)),
         X.drop(['PatientId', 'AppointmentID', 'ScheduledDay', 'AppointmentDay', 'Neighbourhood'], axis=1),
         y,
         scoring=['precision', 'f1', 'roc_auc'],
)

## RandomForestClassifier

[StandardScaler(),
 RandomForestClassifier(class_weight='balanced', max_depth=13, random_state=100)]


### cross_validate

{'fit_time': 3.186,
 'score_time': 0.454,
 'test_f1': 0.344,
 'test_precision': 0.275,
 'test_roc_auc': 0.6}



### confusion matrix

array([[12178,  5464],
       [ 2419,  2044]])



### scores

{'f1_score': 0.341,
 'precision_score': 0.272,
 'roc_auc_score': 0.598}


In [22]:
# custom implicit pipeline

estimate(RandomForestClassifier(random_state=100, class_weight='balanced',
                                n_estimators=100, max_depth=13),
         X,
         y,
         scoring=['precision', 'f1', 'roc_auc'],
         verbose=1,
        )

## RandomForestClassifier

[DebugTransformer(name='before'),
 FeaturerEngineering(name='FeaturerEngineering'),
 DebugTransformer(name='after FeaturerEngineering'),
 ColumnTransformer(n_jobs=-1,
                  transformers=[('minmaxscaler', MinMaxScaler(),
                                 ['Handcap', 'previous_no_show', 'Age',
                                  'days_diff', 'neighbourhood', 'scheduled_day',
                                  'scheduled_hour', 'appointment_day',
                                  'SMS_received', 'Scholarship', 'Hipertension',
                                  'Diabetes', 'Alcoholism'])]),
 DebugTransformer(name='after ColumnTransformer'),
 RandomForestClassifier(class_weight='balanced', max_depth=13, random_state=100)]


### cross_validate

{'fit_time': 6.435,
 'score_time': 0.632,
 'test_f1': 0.451,
 'test_precision': 0.334,
 'test_roc_auc': 0.741}



### fitting

...

before                         transform 	X shape: (22105, 13) 	y shape: None
after FeaturerEngineering      transform 	X shape: (22105, 19) 	y shape: None
after ColumnTransformer        transform 	X shape: (22105, 13) 	y shape: None



### confusion matrix

array([[11389,  6253],
       [ 1338,  3125]])



### scores

{'f1_score': 0.452,
 'precision_score': 0.333,
 'roc_auc_score': 0.74}


In [23]:
# custom implicit pipeline, verbose=2

estimate(RandomForestClassifier(random_state=100, class_weight='balanced',
                                n_estimators=100, max_depth=13),
         X,
         y,
         scoring=['precision', 'f1', 'roc_auc'],
         # remainder='passthrough',
         verbose=2,
        )

## RandomForestClassifier

[DebugTransformer(name='before'),
 FeaturerEngineering(name='FeaturerEngineering'),
 DebugTransformer(name='after FeaturerEngineering', verbose=['transform']),
 ColumnTransformer(n_jobs=-1,
                  transformers=[('minmaxscaler', MinMaxScaler(),
                                 ['Handcap', 'previous_no_show', 'Age',
                                  'days_diff', 'neighbourhood', 'scheduled_day',
                                  'scheduled_hour', 'appointment_day',
                                  'SMS_received', 'Scholarship', 'Hipertension',
                                  'Diabetes', 'Alcoholism'])]),
 DebugTransformer(name='after ColumnTransformer', verbose=['transform']),
 RandomForestClassifier(class_weight='balanced', max_depth=13, random_state=100)]


### cross_validate

{'fit_time': 6.223,
 'score_time': 0.871,
 'test_f1': 0.451,
 'test_precision': 0.334,
 'test_roc_auc': 0.741}



### fitting

...

,count,mean,std,min,25%,50%,75%,max
minmaxscaler__Handcap,22105.0000,0.0057,0.0409,0.0000,0.0000,0.0000,0.0000,1.0000
minmaxscaler__previous_no_show,22105.0000,0.1066,0.0363,0.0000,0.1097,0.1097,0.1097,1.0000
minmaxscaler__Age,22105.0000,0.3287,0.1998,0.0086,0.1638,0.3276,0.4828,1.0000
minmaxscaler__days_diff,22105.0000,0.0575,0.0853,0.0000,0.0000,0.0223,0.0838,1.0000
minmaxscaler__neighbourhood,22105.0000,0.4464,0.1562,0.0000,0.3371,0.4293,0.5046,1.0000
minmaxscaler__scheduled_day,22105.0000,0.3708,0.2769,0.0000,0.2000,0.4000,0.6000,1.0000
minmaxscaler__scheduled_hour,22105.0000,0.3183,0.2144,0.0000,0.1333,0.2667,0.5333,0.9333
minmaxscaler__appointment_day,22105.0000,0.3716,0.2760,0.0000,0.2000,0.4000,0.6000,1.0000
minmaxscaler__SMS_received,22105.0000,0.3240,0.4680,0.0000,0.0000,0.0000,1.0000,1.0000
minmaxscaler__Scholarship,22105.0000,0.0987,0.2983,0.0000,0.0000,0.0000,0.0000,1.0000


### confusion matrix

array([[11389,  6253],
       [ 1338,  3125]])



### scores

{'f1_score': 0.452,
 'precision_score': 0.333,
 'roc_auc_score': 0.74}
